In [ ]:

import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

load DAATA


In [ ]:
TICKER = "^VNINDEX.VN"          
PERIOD = "720d"               
INTERVAL = "60m"              
df = yf.download(TICKER, period=PERIOD, interval=INTERVAL, auto_adjust=True)
df.to_csv('Data_VNIDEX_1H.csv')


đọc data và hiểu

In [ ]:
df = pd.read_csv('Data_VNIDEX_1H.csv')

In [ ]:
df.head()


In [ ]:

df.columns = ['thoi_gian', 'gia_dong_cua', 'gia_cao_nhat', 'gia_thap_nhat', 'gia_mo_cua', 'khoi_luong']

In [ ]:
df['thoi_gian'] = pd.to_datetime(df['thoi_gian'], errors='coerce')
df['thoi_gian'] = df['thoi_gian'].dt.tz_convert('Asia/Ho_Chi_Minh')
df = df.dropna(subset=['thoi_gian']).reset_index(drop=True)

In [ ]:
df = df.drop(columns=['khoi_luong'])

In [ ]:
cols = ['gia_dong_cua', 'gia_cao_nhat', 'gia_thap_nhat', 'gia_mo_cua',]
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

feature

bien dong

In [ ]:
df['bien_do'] = df['gia_cao_nhat'] - df['gia_thap_nhat']
df['ty_le_thay_doi'] = (df['gia_dong_cua'] - df['gia_mo_cua']) / df['gia_mo_cua']

mo hinh nến

In [ ]:
df['rau_tren'] = df['gia_cao_nhat'] - df[['gia_mo_cua', 'gia_dong_cua']].max(axis=1)
df['rau_duoi'] = df[['gia_mo_cua', 'gia_dong_cua']].min(axis=1) - df['gia_thap_nhat']
df['than_nen'] = abs(df['gia_dong_cua'] - df['gia_mo_cua'])
df['la_nen_xanh'] = (df['gia_dong_cua'] > df['gia_mo_cua']).astype(int)
df['ty_le_than_voi_range'] = df['than_nen'] / df['bien_do'].replace(0, np.nan)

xu hương

In [ ]:
df['xu_huong_3'] = df['gia_dong_cua'] - df['gia_dong_cua'].shift(3)
df['xu_huong_5'] = df['gia_dong_cua'] - df['gia_dong_cua'].shift(5)

 Biến động ngắn hạn


In [ ]:
df['do_lech_chuan_3'] = df['gia_dong_cua'].rolling(3).std()
df['do_lech_chuan_5'] = df['gia_dong_cua'].rolling(5).std()

cuốn chíu

In [ ]:
df['trung_vi_5h'] = df['gia_dong_cua'].rolling(5).median()
df['max_5h'] = df['gia_dong_cua'].rolling(5).max()
df['min_5h'] = df['gia_dong_cua'].rolling(5).min()
df['do_lech_trung_vi'] = df['gia_dong_cua'] - df['trung_vi_5h']
df['skew_5h'] = df['gia_dong_cua'].rolling(5).skew()
df['kurt_5h'] = df['gia_dong_cua'].rolling(5).kurt()

khỏng trống giá

In [ ]:
df['gap_len'] = ((df['gia_mo_cua'] - df['gia_dong_cua'].shift(1)) / df['gia_dong_cua'].shift(1)) > 0.01
df['gap_xuong'] = ((df['gia_mo_cua'] - df['gia_dong_cua'].shift(1)) / df['gia_dong_cua'].shift(1)) < -0.01

lag

In [ ]:
for i in range(1, 11):
    df[f'gia_dong_cua_lag_{i}'] = df['gia_dong_cua'].shift(i)


In [ ]:
df = df.dropna().reset_index(drop=True)

label

In [ ]:
print(df.columns.tolist())


In [ ]:
n = 20
df['percent_change_future'] = (df['gia_dong_cua'].shift(-n) - df['gia_dong_cua']) / df['gia_dong_cua']
df['label'] = 0
df.loc[df['percent_change_future'] > 0.01, 'label'] = 2
df.loc[df['percent_change_future'] < -0.01, 'label'] = 1


In [ ]:
print(df['label'].value_counts())


model

In [ ]:
X = df.drop(columns=['thoi_gian', 'label','percent_change_future'], errors='ignore')
y = df['label']
X = X.dropna()
y = y.loc[X.index]

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = xgb.XGBClassifier(
    objective='binary:logistic' if y.nunique() == 2 else 'multi:softmax',
    num_class=y.nunique() if y.nunique() > 2 else None,
    eval_metric='mlogloss' if y.nunique() > 2 else 'logloss',
    use_label_encoder=False,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))



In [ ]:
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)
print("Train label:", y_train.value_counts())
print("Test label:", y_test.value_counts())


In [ ]:
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
y_proba = model.predict_proba(X_test)

auc_score = roc_auc_score(y_test_bin, y_proba, multi_class='ovr')  #

In [ ]:
auc_score